In [15]:
# Bildverarbeitung & Klassifikation (PIL statt OpenCV)
import os
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import matplotlib.pyplot as plt

# CSV einlesen
df = pd.read_csv("metadata.csv")

# Nur Bilder verwenden, die tatsächlich vorhanden sind
df = df[df["image_name"].apply(lambda x: os.path.exists(os.path.join("images", x)))]

# Top-Marken (mind. 50 Bilder)
top_brands = df["brand"].value_counts()[df["brand"].value_counts() >= 50].index
df = df[df["brand"].isin(top_brands)].reset_index(drop=True)

# Labels kodieren
le = LabelEncoder()
df["label"] = le.fit_transform(df["brand"])

# Bilder laden mit PIL
images = []
labels = []

for _, row in df.iterrows():
    img_path = os.path.join("images", row["image_name"])
    try:
        img = Image.open(img_path).convert("RGB")
        img = img.resize((128, 128))
        img_array = np.array(img) / 255.0
        images.append(img_array)
        labels.append(row["label"])
    except:
        continue  # falls ein Bild beschädigt ist

X = np.array(images)
y = to_categorical(labels)

# Train/Test-Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# CNN-Modell bauen
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(y.shape[1], activation='softmax')
])

# Kompilieren & Trainieren
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Bewertung & Plot
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc * 100:.2f}%")

# Genauigkeits-Plot
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.title('Accuracy')
plt.legend()
plt.show()


ModuleNotFoundError: No module named 'numpy.typing'

In [ ]:
# CSV laden
df = pd.read_csv("metadata.csv")

# Bilderordner
image_folder = "images"

# Nur Bilder verwenden, die vorhanden sind
df = df[df["image_name"].apply(lambda x: os.path.isfile(os.path.join(image_folder, x)))]

# Nur Marken mit genügend Bildern
top_brands = df["brand"].value_counts()[df["brand"].value_counts() >= 50].index
df = df[df["brand"].isin(top_brands)].reset_index(drop=True)

# Label-Encoding
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["brand"])

In [ ]:
images = []
labels = []

for _, row in df.iterrows():
    img_path = os.path.join(image_folder, row["image_name"])
    img = cv2.imread(img_path)
    img = cv2.resize(img, (128, 128)) / 255.0
    images.append(img)
    labels.append(row["label"])

X = np.array(images)
y = to_categorical(labels)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(y.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


In [ ]:
# Genauigkeit anzeigen
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc * 100:.2f}%")

# Trainingsverlauf
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.title('Accuracy')
plt.legend()
plt.show()
